## Import Necessary Libraries

In [256]:
import pandas as pd
import numpy as np
import uuid
import json
from pathlib import Path

## WORKING IN NEW STRATEGY...

In [ ]:
SHORT_WINDOW = 5
LONG_WINDOW = 15

MOMENTUM_WINDOW = 3

VOLATILITY_THRESHOLD = 1

class DefaultStrategy:
    def __init__(self):
        self.initialized = False
        
        # Price history for each pair - this maintains state between calls
        self.price_history = {
            "token_1/fiat": [],
            "token_2/fiat": [],
            "token_1/token_2": []
        }
        
        self.short_window = SHORT_WINDOW
        self.long_window = LONG_WINDOW

        self.momentum_window = MOMENTUM_WINDOW
        
        # Volatility threshold for signals
        self.threshold = VOLATILITY_THRESHOLD

    def on_data(self, market_data, balances):

        """Process market data and current balances to make trading decisions.
        
        Args:
            market_data: Dictionary of {pair: tick_data} containing market data for each pair
            balances: Dictionary of {currency: amount} containing current balances
        
        Returns:
            Trading signal dict {pair, side, qty} or None
        """


        orders = []
        
        # Update price history for each pair
        for pair, data in market_data.items():

            if pair in self.price_history:
                self.price_history[pair].append(data["close"])
                if len(self.price_history[pair]) > self.long_window + 1:
                    self.price_history[pair] = self.price_history[pair][-self.long_window-1:]
                
                # Limit history length


        
        
        # Wait until we have enough data points
        for prices in self.price_history.values():
            if len(prices) < self.long_window:
                return orders
        
        # Initialize flag for trading
        if not self.initialized:
            self.initialized = True
            return orders
        
        # Check for trading opportunities in token_1/fiat
        if "token_1/fiat" in market_data:
            prices = self.price_history["token_1/fiat"]
            price = prices[-1]

            short_prices = self.price_history["token_1/fiat"][-self.short_window:]
            long_prices = self.price_history["token_1/fiat"][-self.long_window:]

            previous_short_ma = np.mean(prices[-self.short_window-1:-1])
            previous_long_ma = np.mean(prices[-self.long_window-1:-1])

            momentum = prices[-1] - prices[-self.momentum_window]

            short_ma = np.mean(short_prices)
            long_ma = np.mean(long_prices)

            mu, sigma = np.mean(prices), np.std(prices)
            
            base_qty = 0.01
            max_fraction = 0.1  # 10% of fiat
            vol_factor = 1 - min(sigma / price, 0.5)
            qty = min(base_qty * vol_factor, balances["fiat"] * max_fraction / price)
            qty = max(0.005, qty)  # never trade less than 0.005
            
            booleanMABuy = previous_short_ma <= previous_long_ma and short_ma > long_ma
            booleanMASell = previous_short_ma >= previous_long_ma and short_ma < long_ma

            if   booleanMABuy and momentum > 0:
                # Get fee from market_data if available, otherwise use default
                fee = market_data["fee"]

                required_fiat = qty * price * (1 + fee)
                if balances["fiat"] >= required_fiat:
                    orders.append({"pair": "token_1/fiat", "side": "buy", "qty": qty})
            
            elif booleanMASell and momentum < 0:
                # Sell token_1 for fiat if we have enough token_1
                qty = min(qty, balances["token_1"])  # Adjust qty based on available balance
                if qty > 0:
                    orders.append({"pair": "token_1/fiat", "side": "sell", "qty": qty})
        
        # Check for trading opportunities in token_2/fiat
        if "token_2/fiat" in market_data:
            prices = self.price_history["token_2/fiat"]
            price = prices[-1]

            short_prices = self.price_history["token_2/fiat"][-self.short_window:]
            long_prices = self.price_history["token_2/fiat"][-self.long_window:]

            previous_short_ma = np.mean(prices[-self.short_window-1:-1])
            previous_long_ma = np.mean(prices[-self.long_window-1:-1])

            momentum = prices[-1] - prices[-self.momentum_window]

            short_ma = np.mean(short_prices)
            long_ma = np.mean(long_prices)

            mu, sigma = np.mean(prices), np.std(prices)
            
            base_qty = 0.01
            max_fraction = 0.1  # 10% of fiat
            vol_factor = 1 - min(sigma / price, 0.5)
            qty = min(base_qty * vol_factor, balances["fiat"] * max_fraction / price)
            qty = max(0.005, qty)  # never trade less than 0.005

            booleanMABuy = previous_short_ma <= previous_long_ma and short_ma > long_ma
            booleanMASell = previous_short_ma >= previous_long_ma and short_ma < long_ma

            # Buy token_2 with fiat if we have enough fiata
            if booleanMABuy and momentum > 0:
                # Get fee from market_data if available, otherwise use default
                fee = market_data["fee"]

                required_fiat = qty * price * (1 + fee)
                if balances["fiat"] >= required_fiat:
                    orders.append({"pair": "token_2/fiat", "side": "buy", "qty": qty})
            
            elif booleanMASell and momentum < 0:
                # Sell token_2 for fiat if we have enough token_2
                qty = min(qty, balances["token_2"])  # Adjust qty based on available balance
                if qty > 0:
                    orders.append({"pair": "token_2/fiat", "side": "sell", "qty": qty})
        
        # Check for arbitrage opportunities with token_1/token_2
        if all(pair in market_data for pair in ["token_1/fiat", "token_2/fiat", "token_1/token_2"]):
            token1_price = market_data["token_1/fiat"]["close"]
            token2_price = market_data["token_2/fiat"]["close"]
            token1_token2_price = market_data["token_1/token_2"]["close"]
            
            # Calculate implied token_1/token_2 price
            implied_token1_token2 = token1_price / token2_price
            
            # If actual token_1/token_2 price is significantly lower than implied
            if token1_token2_price < implied_token1_token2 * 0.995:
                # Buy token_1 with token_2 (if we have token_2)
                qty_token1 = 0.01
                # Get fee from market_data if available, otherwise use default
                fee = market_data["fee"]
                required_token2 = qty_token1 * token1_token2_price * (1 + fee)
                if balances["token_2"] >= required_token2:
                    orders.append({"pair": "token_1/token_2", "side": "buy", "qty": qty_token1})
            
            # If actual token_1/token_2 price is significantly higher than implied
            elif token1_token2_price > implied_token1_token2 * 1.005:
                # Sell token_1 for token_2 (if we have token_1)
                qty_token1 = min(0.01, balances["token_1"])  # Adjust qty based on available balance
                if qty_token1 > 0:
                    orders.append({"pair": "token_1/token_2", "side": "sell", "qty": qty_token1})
        
        print(f"Balance: {balances["fiat"] + balances["token_1"]*self.price_history["token_1/fiat"][-1] + balances["token_2"]*self.price_history["token_2/fiat"][-1]}")
        return orders

strategy = DefaultStrategy()

## Submission File Generation

PARTICIPANTS SHOULD NOT MODIFY THIS FUNCTION.
It is part of the evaluation process to ensure the strategy works as intended.
This function is used to validate the strategy. It runs a backtest on the strategy's performance using historical market data.

In [258]:
class Trader:
    """Trader supporting multiple trading pairs and currencies."""

    def __init__(self, balances, fee):
        # Initialize balances for each currency
        self.balances = balances

        # Track market prices for each pair
        self.prices = {
            "token_1/fiat": None,
            "token_2/fiat": None,
            "token_1/token_2": None
        }

        # First and last prices for reporting
        self.first_prices = {
            "token_1/fiat": None,
            "token_2/fiat": None,
            "token_1/token_2": None
        }

        # Store the first update timestamp for each pair
        self.first_update = {
            "token_1/fiat": False,
            "token_2/fiat": False,
            "token_1/token_2": False
        }

        # Track portfolio value history
        self.equity_history = 0.0
        self.turnover = 0.0
        self.trade_count = 0
        self.total_fees_paid = 0.0  # Track total fees paid

        # Trading fee
        self.fee = fee

    def update_market(self, pair, price_data):
        """Update market prices for a specific trading pair"""
        # Store the updated price
        self.prices[pair] = price_data["close"]

        # Store first price for each pair (for reporting)
        if not self.first_update[pair]:
            self.first_prices[pair] = price_data["close"]
            self.first_update[pair] = True

        # Calculate total portfolio value (in fiat)
        equity = self.calculate_portfolio_value()
        self.equity_history.append(equity)

    def calculate_portfolio_value(self):
        """Calculate total portfolio value in fiat currency"""
        value = self.balances["fiat"]

        # Add token_1 value if we have price data
        if self.prices["token_1/fiat"] is not None:
            value += self.balances["token_1"] * self.prices["token_1/fiat"]

        # Add token_2 value if we have price data
        if self.prices["token_2/fiat"] is not None:
            value += self.balances["token_2"] * self.prices["token_2/fiat"]
        # If token_2/fiat price not available but token_1/fiat and token_1/token_2 are available
        elif self.prices["token_1/fiat"] is not None and self.prices["token_1/token_2"] is not None:
            token2_value_in_token1 = self.balances["token_2"] / self.prices["token_1/token_2"]
            value += token2_value_in_token1 * self.prices["token_1/fiat"]

        return value

    def execute(self, order):
        """Execute a trading order across any supported pair"""
        pair = order["pair"]  # e.g., "token_1/fiat"
        side = order["side"]  # "buy" or "sell"
        qty = float(order["qty"])

        # Split the pair into base and quote currencies
        base, quote = pair.split("/")

        # Get current price for the pair
        price = self.prices[pair]
        if price is None:
            return  # Can't trade without a price

        executed = False

        if side == "buy":
            # Calculate total cost including fee
            base_cost = qty * price
            fee_amount = base_cost * self.fee
            total_cost = base_cost + fee_amount

            # Check if we have enough of the quote currency
            if self.balances[quote] >= total_cost:
                # Deduct quote currency (e.g., fiat)
                self.balances[quote] -= total_cost

                # Add base currency (e.g., token_1)
                self.balances[base] += qty

                # Track turnover and fees
                self.turnover += total_cost
                self.total_fees_paid += fee_amount
                executed = True

        elif side == "sell":
            # Check if we have enough of the base currency
            if self.balances[base] >= qty:
                # Calculate proceeds after fee
                base_proceeds = qty * price
                fee_amount = base_proceeds * self.fee
                net_proceeds = base_proceeds - fee_amount

                # Add quote currency (e.g., fiat)
                self.balances[quote] += net_proceeds

                # Deduct base currency (e.g., token_1)
                self.balances[base] -= qty

                # Track turnover and fees
                self.turnover += base_proceeds
                self.total_fees_paid += fee_amount
                executed = True

        # Count successful trades
        if executed:
            self.trade_count += 1

In [259]:
def run_backtest(combined_data: pd.DataFrame, fee: float, balances: dict[str, float]) -> pd.DataFrame:
    """Run a backtest with multiple trading pairs.

    Args:
        submission_dir: Path to the strategy directory
        combined_data: DataFrame containing market data for multiple pairs
        fee: Trading fee (in basis points, e.g., 2 = 0.02%)
        balances: Dictionary of {pair: amount} containing initial balances
    """
    # Record initial balances for display
    trader = Trader(balances, fee)

    initial_balances = balances.copy()

    # Initialize prices with first data point for each pair
    combined_data.sort_values("timestamp", inplace=True)
    first_prices = {k: df.iloc[0]['close'] for k, df in combined_data.groupby("symbol")}

    # Calculate true initial portfolio value including all assets
    initial_portfolio_value = initial_balances["fiat"]
    if "token_1/fiat" in first_prices and initial_balances["token_1"] > 0:
        initial_portfolio_value += initial_balances["token_1"] * first_prices["token_1/fiat"]
    if "token_2/fiat" in first_prices and initial_balances["token_2"] > 0:
        initial_portfolio_value += initial_balances["token_2"] * first_prices["token_2/fiat"]

    trader.equity_history = [initial_portfolio_value]
    # Combine all dataframes and sort by timestamp
    result = pd.DataFrame(
        columns=["id", "timestamp", "pair", "side", "qty"],
    )

    # Process data timestamp by timestamp
    for timestamp, group in combined_data.groupby('timestamp'):
        # Update prices for each pair in this timestamp
        market_data = {
            "fee": fee,
        }
        for _, row in group.iterrows():
            pair = row['symbol']
            data_dict = row.to_dict()
            # Add fee information to market data so strategies can access it
            market_data[pair] = data_dict
            trader.update_market(pair, data_dict)
        
        # Get strategy decision based on all available market data and current balances
        orders = strategy.on_data(market_data, balances)

        # Handle list of orders
        for order in orders:
            trader.execute(order)
            order["timestamp"] = timestamp
            order["id"] = str(uuid.uuid4())
            result = pd.concat([result, pd.DataFrame([order])], ignore_index=True)

    return result

In [260]:
with open("./input/hyperparameters.json") as f:
    HYPERPARAMETERS = json.load(f)
    
FEE = HYPERPARAMETERS.get("fee", 3.0)/10000.0  # Convert to decimal
BALANCE_FIAT = HYPERPARAMETERS.get("fiat_balance", 10000.0)
BALANCE_TOKEN1 = HYPERPARAMETERS.get("token1_balance", 0.0)
BALANCE_TOKEN2 = HYPERPARAMETERS.get("token2_balance", 0.0)
OUTPUT = "submission.csv"

combined_data = pd.read_csv("./input/test.csv")

# Run the backtest on the provided test data with a fee of 0.02% and initial balances of 10,000 fiat, and 0 token_1 and token_2
result = run_backtest(combined_data, FEE, {
    "fiat": BALANCE_FIAT,
    "token_1": BALANCE_TOKEN1,
    "token_2": BALANCE_TOKEN2,
})

# Output the backtest result to a CSV file for submission
result.to_csv(OUTPUT, index=False)

Balance: 1802502.4
Balance: 1801580.9
Balance: 1800842.8151521191
Balance: 1801021.7021641887
Balance: 1801392.7928698552
Balance: 1801294.5613058237
Balance: 1801009.8843696632
Balance: 1801151.5323413361
Balance: 1801587.612511117
Balance: 1801717.6083034482
Balance: 1801990.9082763582
Balance: 1802357.6062362154
Balance: 1801651.223744798
Balance: 1801618.5237136981
Balance: 1801769.923816523
Balance: 1801387.7236189588
Balance: 1801123.182649353
Balance: 1801342.5507819161
Balance: 1801234.324683189
Balance: 1801521.6562907211
Balance: 1801819.9610377275
Balance: 1801672.3324973446
Balance: 1801006.022711176
Balance: 1801747.9571822393
Balance: 1801550.1251558508
Balance: 1801911.2910284766
Balance: 1802079.821215819
Balance: 1802063.9202142332
Balance: 1802247.892404973
Balance: 1802045.2922654906
Balance: 1802377.3925780528
Balance: 1802912.1933305915
Balance: 1803111.89354733
Balance: 1803297.993861041
Balance: 1803222.7939131926
Balance: 1803219.6938099135
Balance: 1803369.8938

C:\Users\rafey\AppData\Local\Temp\ipykernel_1972\3259639209.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, pd.DataFrame([order])], ignore_index=True)


Balance: 1833115.8213754054
Balance: 1832942.4847536648
Balance: 1833187.916263884
Balance: 1833718.4043155708
Balance: 1833474.0207319409
Balance: 1832889.8210049947
Balance: 1832683.709813966
Balance: 1832459.8872225666
Balance: 1831834.4645342543
Balance: 1831655.4740094258
Balance: 1832193.4311742662
Balance: 1832739.162413389
Balance: 1833548.0826721727
Balance: 1833998.5808146023
Balance: 1834356.338815027
Balance: 1832559.7945786084
Balance: 1831826.1731208276
Balance: 1831751.5970384832
Balance: 1831221.5816690922
Balance: 1831349.7898774105
Balance: 1830322.7666733107
Balance: 1832088.2983616919
Balance: 1832787.9261512565
Balance: 1832209.612607541
Balance: 1832493.3672705467
Balance: 1832801.2360428893
Balance: 1833373.729291514
Balance: 1832856.9274613694
Balance: 1832652.6332944974
Balance: 1832686.6503306273
Balance: 1833235.9735586725
Balance: 1835257.0258245885
Balance: 1834537.782461796
Balance: 1835065.4066641992
Balance: 1834889.4832082747
Balance: 1834719.194277335
